In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Download helper functions script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2023-07-15 16:12:03--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-07-15 16:12:03 (103 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [2]:
# Download data (same as from Kaggle)
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2023-07-15 16:12:04--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.194.128, 142.250.4.128, 74.125.200.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.194.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K   535KB/s    in 1.1s    

2023-07-15 16:12:06 (535 KB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [3]:

import tensorflow_hub as hub
import re
import string
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

# Load data
df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
df.head()


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [6]:
# Convert text to lowercase
df['text'] = df['text'].str.lower()

# Remove punctuation
df['text'] = df['text'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

# Remove stopwords
stop_words = set(stopwords.words('english'))
df['text'] = df['text'].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))

In [7]:
import nltk
from nltk.corpus import wordnet
import random

def get_synonyms(word):
    synonyms = []
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.append(lemma.name())
    return synonyms

def synonym_replacement(sentence):
    words = sentence.split()
    new_words = words.copy()
    random_word_list = list(set([word for word in words if word not in stop_words]))
    random_word = random.choice(random_word_list)
    synonyms = get_synonyms(random_word)
    if len(synonyms) >= 1:
        synonym = random.choice(synonyms)
        new_words = [synonym if word == random_word else word for word in new_words]
    sentence = ' '.join(new_words)
    return sentence

# Apply the synonym_replacement to augment your text data
df['text'] = df['text'].apply(synonym_replacement)


In [8]:


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['target'], test_size=0.2, random_state=42)


In [9]:
# # Define the model
# model = tf.keras.Sequential([
#     hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4", trainable=False),
#     tf.keras.layers.Dense(256, activation='relu'),
#     tf.keras.layers.Dense(1, activation='sigmoid')
# ])

# model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
#               optimizer=tf.keras.optimizers.Adam(),
#               metrics=['accuracy'])

# # Train the model
# history = model.fit(np.array(X_train), np.array(y_train), epochs=10, validation_data=(np.array(X_test), np.array(y_test)))


In [10]:
# # Adding a LSTM layer
# model_1 = tf.keras.Sequential([
#     hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4", trainable=False),
#     tf.keras.layers.Reshape((1, 512)),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
#     tf.keras.layers.Dense(64, activation='relu'),
#     tf.keras.layers.Dense(1, activation='sigmoid')
# ])


In [11]:
# # Change optimizer and learning rate
# model_1.compile(loss=tf.keras.losses.BinaryCrossentropy(),
#               optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
#               metrics=['accuracy'])

# history = model_1.fit(np.array(X_train), np.array(y_train), epochs=10, validation_data=(np.array(X_test), np.array(y_test)))


In [12]:

# model_1.evaluate(np.array(X_test), np.array(y_test))


In [13]:
model_2 = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4", trainable=False),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model_2.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

history_2 = model_2.fit(np.array(X_train), np.array(y_train), epochs=10, validation_data=(np.array(X_test), np.array(y_test)))


Epoch 1/10
191/191 [==============================] - 12s 19ms/step - loss: 0.5110 - accuracy: 0.7755 - val_loss: 0.4496 - val_accuracy: 0.8004
Epoch 2/10
191/191 [==============================] - 2s 12ms/step - loss: 0.4350 - accuracy: 0.8025 - val_loss: 0.4425 - val_accuracy: 0.8011
Epoch 3/10
191/191 [==============================] - 3s 13ms/step - loss: 0.4196 - accuracy: 0.8103 - val_loss: 0.4387 - val_accuracy: 0.7971
Epoch 4/10
191/191 [==============================] - 2s 12ms/step - loss: 0.4106 - accuracy: 0.8125 - val_loss: 0.4400 - val_accuracy: 0.7991
Epoch 5/10
191/191 [==============================] - 3s 18ms/step - loss: 0.3961 - accuracy: 0.8225 - val_loss: 0.4347 - val_accuracy: 0.8076
Epoch 6/10
191/191 [==============================] - 2s 13ms/step - loss: 0.3872 - accuracy: 0.8263 - val_loss: 0.4346 - val_accuracy: 0.8076
Epoch 7/10
191/191 [==============================] - 4s 21ms/step - loss: 0.3750 - accuracy: 0.8342 - val_loss: 0.4354 - val_accuracy: 0.799

In [22]:
test_df['text'][1]

'Heard about #earthquake is different cities, stay safe everyone.'

In [24]:
for i in range(10):
  print(f"Text: {test_df['text'][i]}\n")
  print("Disaster" if np.round(model_2.predict(np.array([test_df['text'][i]]), verbose = 0)) >0 else "Not Disaster")


Text: Just happened a terrible car crash

Disaster
Text: Heard about #earthquake is different cities, stay safe everyone.

Disaster
Text: there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all

Disaster
Text: Apocalypse lighting. #Spokane #wildfires

Disaster
Text: Typhoon Soudelor kills 28 in China and Taiwan

Disaster
Text: We're shaking...It's an earthquake

Disaster
Text: They'd probably still show more life than Arsenal did yesterday, eh? EH?

Not Disaster
Text: Hey! How are you?

Not Disaster
Text: What a nice hat?

Not Disaster
Text: Fuck off!

Not Disaster
